# This is an example notebook to show how our code can be used for senario modelling

Let us look at how to model two scenarios:

1. Business-As-Usual - A linear trend of the demand from historical data is used to simulate the future land use changes.
2. Growth-As-Usual - A constant demand, calculated from the last data available, is continued further into the future

## Imports

We first import the required modules

In [4]:
import os
from SuSceMod.simulation import logger, data_utilities, input_handling, scenarios

## Setup of the simulation

Need to first add the paths to the input and output folders

In [5]:
parent_folder = '.'
input_folder = './input_rasters/'

output_folder = './output/'
os.makedirs(output_folder, exist_ok=True)

suitability_maps_folder = './suitability_maps/'

In [6]:
# log_file_path = output_folder + f'/BAU_simulation.log'
# logger.setup_logger(log_file_path, log_time=False)

In [7]:
initial_year = 2020
final_year = 2050

In [8]:
prob_years = [2010, 2020]
change_years = [2010, 2020]

In [9]:
initial_built_up_map = data_utilities.load_raster_data(input_folder + f'BU_{initial_year}.tif') 

Path to the suitability maps must be added, to guide the allocation. These can be outputs of ML models likes MultiNomial Logistic regression in our case 

In [10]:
suitability_maps_path = suitability_maps_folder + f"{str(prob_years[0])[-2:]}_{str(prob_years[1])[-2:]}/"
prob_maps_dict, prob_map_paths = input_handling.load_prob_maps(suitability_maps_path, prob_years)

Give the paths to the maps to calculate the change maps

In [11]:
initial_demand_map = input_folder + f'BU_{change_years[0]}.tif'
final_demand_map   = input_folder + f'BU_{change_years[1]}.tif'

change_maps_dict, classes = input_handling.calculate_change_maps(
    initial_demand_map, final_demand_map
)

In [12]:
annual_rates, period_counts, crosstab_df = input_handling.calculate_change_rates(
    change_maps_dict, change_years
)

## Running the simulation

In [13]:
simulated_maps, output_paths, class_counts = scenarios.simulate_growth_BAU(
    initial_year,
    final_year,
    initial_built_up_map, 
    prob_maps_dict,
    change_years,
    annual_rates,
    output_folder,
    parent_folder
)


###################################################################

Current year: 2020
Simulating for year: 2021


D:\Codes\Scripts_CA\CA_MCMC\TO UPLOAD\Sept_2025\SuSceMod\SuSceMod\simulation\core_functions.py:14: RuntimeWarning: invalid value encountered in cast
  a = built_up_map.astype(np.int32, copy=False)
D:\Codes\Scripts_CA\CA_MCMC\TO UPLOAD\Sept_2025\SuSceMod\SuSceMod\simulation\core_functions.py:61: RuntimeWarning: invalid value encountered in cast
  donor_mask = (valid & (a.astype(np.int32) == i))
D:\Codes\Scripts_CA\CA_MCMC\TO UPLOAD\Sept_2025\SuSceMod\SuSceMod\simulation\core_functions.py:108: RuntimeWarning: invalid value encountered in cast
  a_int = a.astype(np.int32, copy=False)



------------------------------------------------------

Year 2021: Updated built-up map summary (value: count):
{0.0: 1380924, 1.0: 77153, 2.0: 168756, 3.0: 63287, 'nan': 2004749}

###################################################################

Current year: 2021
Simulating for year: 2022

------------------------------------------------------

Year 2022: Updated built-up map summary (value: count):
{0.0: 1380005, 1.0: 77063, 2.0: 169305, 3.0: 63747, 'nan': 2004749}

Changes from previous year summary (value: count):
{0: -919, 1: -90, 2: 549, 3: 460}

###################################################################

Current year: 2022
Simulating for year: 2023

------------------------------------------------------

Year 2023: Updated built-up map summary (value: count):
{0.0: 1379086, 1.0: 76973, 2.0: 169854, 3.0: 64207, 'nan': 2004749}

Changes from previous year summary (value: count):
{0: -919, 1: -90, 2: 549, 3: 460}

######################################################

In [15]:
counts_csv_file = os.path.join(output_folder, f"class_counts_{initial_year}_{final_year}.csv")
data_utilities.save_counts_to_csv(class_counts, counts_csv_file)

In [16]:
template_raster_path = input_folder + f'BU_{change_years[0]}.tif'

In [18]:
for year in simulated_maps.keys():
    data_utilities.writeraster(template_raster_path, output_paths[year], simulated_maps[year])